In [31]:
from mesa import Agent, Model
from mesa.time import StagedActivation
import random
import numpy as np

random.seed(42)

In [32]:
## Parameters 
N = 10
prop_adversaries = .2
coin_toss = .9
rounds = 20

In [33]:
class MyModel(Model):
    def __init__(self, N, coin_toss):
        
        super().__init__()

        assert N > 0, "N must be greater than 0."
        self.num_agents = N

        ## Staged activation to determine which round
        ## And whose turn to be the chooser
        self.schedule = StagedActivation(self, stage_list=["choose", "vote", "act", "update"])
        self.round = 0 
        self.whose_turn = self.round % N

        ## if anyone was chosen
        self.chose = False
        ## who is the chosen, and -1 flags is that no one was
        self.chosen = []

        ## voting record of across the rounds
        self.voting_record = []
        ## voting record of the current round
        self.votes_round = []

        ## Action
        self.action_record = []
        self.action_round = [-1,-1]

        ## Outcome 
        self.outcome = []

        ## Coin Toss
        self.coin_toss = coin_toss
        
        # Change this up a little
        for i in range(self.num_agents):  # Example: creating 10 agents
            agent = DummyAgent(i, self)
            self.schedule.add(agent)

        

    ## CONVENION :: check_* are methods for agents to check certain aspects of the model
    
    """
    CHECKING TURNS and CHOOSER-CHOSEN

    These methods concern the first stage of the model
    where the agents choose or are chosen
    """
    def check_whose_turn(self):
        return self.whose_turn
    
    def check_who_was_chosen(self):
        return self.chosen[self.round]
    
    # check all choose chosen pairs
    def check_historical_choose_chosen_pairs(self):
        
        chooser = [ i % self.num_agents for i in list(range(0, self.round)) ]
        chosen = self.chosen

        choose_chosen_pair = list(zip(chooser, chosen))

        return choose_chosen_pair
    
    """
    CHECKING VOTING

    These methods concern information of voting outcomes
    historical and current
    """
    def check_historical_voting_records(self):
        return self.voting_record
    
    def check_current_voting_record(self):
        return self.votes_round
    
    def check_agent_voting_record(self, agent):
        return [ i[agent] for i in self.voting_record ]
    
    def check_vote_pass(self):

        votes = self.votes_round

        if sum(votes)/len(votes) > 0.5:
            return True
        else:
            return False
    
    """
    CHECKING OUTCOME

    These methods concern information of outcomes
    historical and current
    """
    def check_outcome_current_round(self):
        return self.outcome[-1]
    
    def check_outcome(self):
        return self.outcome
        
    def toss_coin(self):
        # Using random.choices() for a single selection
        toss = random.choices([0,1], weights=[self.coin_toss, 1 - self.coin_toss], k=1)[0]
        return toss
    
    """
    DETERMINE OUTCOME

    Determine Outcomes from Actions
    """
    def outcome_determination(self):
        ## determine which team won in given round
        ## 1 = Good team wins, 0 = Bad team wins
        if self.action_round[0] == 1 and self.action_round[1] == 1:
            self.outcome.append(1)
        elif self.action_round[0] == -1 and self.action_round[1] == -1:
            self.outcome.append(self.toss_coin())
        else:
            self.outcome.append(0)

   
    def step(self):
        self.schedule.step()
        self.round += 1
        
        ## Reset Parameters
        self.whose_turn = self.round % N
        self.chose = False
        self.voting_record.append(self.votes_round)
        self.votes_round = []
        self.action_record.append(self.action_round)
        self.action_round = [-1,-1]



In [35]:
model = MyModel(N, coin_toss)

for i in range(rounds):
    model.step()

In [36]:
model.action_record

[[1, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 0],
 [1, 0],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1]]

In [37]:
model.check_outcome()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [38]:
# Example lists
items = ['apple', 'banana', 'cherry']
probabilities = [0.1, 0.6, 0.3]

# Using random.choices() for a single selection
selected_item = random.choices([0,1], weights=[coin_toss, 1-coin_toss], k=1)[0]
selected_item

0

In [6]:
import random 
from MaxAgents import MaxRandomAgent
from MinAgents import MinRandomAgent
N = 100
prop = .5

random.choices([MinRandomAgent,MaxRandomAgent], weights=[prop, 1 - prop], k=N)

agents

[MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MaxAgents.MaxRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 MinAgents.MinRandomAgent,
 